## シミュレーションを流すノートブック

In [ ]:
#path
using CUDA
using EzXML
using MDToolbox
using Statistics
device!(1)
pdb_filepath = "./alanine-dipeptide-nowater.pdb" #pdbのパス
top = readpdb(pdb_filepath)
ff_start_filepath  = "./data/amber14/protein.ff14SB.xml" #startのffのパス
out_dir = "./target" #出力するディレクトリ
isdir(out_dir) || mkdir(out_dir)

#setting
njobs = 1 #並列して流すシミュレーションの数
nsteps = 2_500_000_000 #シミュレーションのステップ数
gpu_id = "n4"

## シミュレーション

In [ ]:
# 一つのシミュレーションを流すジョブスクリプト
# ff_filepath　　　  :xml formatの力場ファイルのパス
# out_traj_dir      :アウトプットするディレクトリのパス
# i                 :流すシミュレーションの通し番号

function run_job(ff_filepath, traj_filepath, log_filepath, i)
    sbatch_file = "temp_$(i).sh"
    
    open(sbatch_file, "w") do of
        println(of, "#!/bin/bash")
        println(of, "#SBATCH -p all")
        println(of, "#SBATCH -J sim$(i) # job name")
        println(of, "#SBATCH -n 1  # num of total mpi processes")
        println(of, "#SBATCH -c 1  # num of threads per mpi processes")
        println(of, "#SBATCH -o $(log_filepath)")
        println(of, "export CUDA_VISIBLE_DEVICES=$(i)")
        #println(of, "python sim.py $(pdb_filepath) $(ff_filepath) $(traj_filepath) $(nsteps)")
        println(of, "time python sim.py $(pdb_filepath) $(ff_filepath) $(traj_filepath) $(nsteps)")
    end
    sleep(5)
    run(`sbatch --gpus-per-node=1 -w $(gpu_id) $(sbatch_file)`)
    #run(`sbatch -w $(gpu_id) $(sbatch_file)`)
    sleep(5)
    rm(sbatch_file)
end

In [ ]:
ff_filepath = ff_start_filepath

traj_filepath = joinpath(out_dir, "traj_target")
log_filepath = joinpath(out_dir, "sim_target.log")

run_job(ff_filepath, traj_filepath, log_filepath, 1)